# Prepare Stage

## Goals for this notebook:

    1. Split data
    2. Handle missing and nulls
    3. Handle outliers
    4. Encode variables
    5. Scale data
    6. Create new features

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare

In [2]:
# Acquire data from acquire.py
telco = acquire.get_telco_data()
# function to change the types of variables goes here, or hopefully its covered in the acquire file

In [3]:
telco.head()

,contract_type_id,internet_service_type_id,payment_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,payment_type,internet_service_type,contract_type
0,1,1,2,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Mailed check,DSL,Month-to-month
1,1,1,4,0013-MHZWF,Female,0,No,Yes,9,Yes,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Credit card (automatic),DSL,Month-to-month
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,...,No,No,No,Yes,48.2,340.35,No,Electronic check,DSL,Month-to-month
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,...,No,No,No,Yes,25.1,25.1,Yes,Electronic check,DSL,Month-to-month
4,1,1,3,0032-PGELS,Female,0,Yes,Yes,1,No,...,No,No,No,No,30.5,30.5,Yes,Bank transfer (automatic),DSL,Month-to-month


### Split data

In [5]:
train, test = prepare.split_my_data(telco, 0.8)
train, valid = prepare.split_my_data(train, 0.8)

### Handle missing and null values

In [ ]:
# The 11 null values from tenure go here, but that also means the total_charges associated with them should be addressed



### Handle outliers

In [ ]:
# Identify outliers



### Encode variables

> variables with Yes or No will be converted to 1 or 0

In [6]:
def boolean_labeler(df, col):
    le = preprocessing.LabelEncoder()
    df[f'{col}_enc'] = le.fit_transform(df[col])
    return df

In [7]:
train = prepare.boolean_labeler(train, 'gender')
valid = prepare.boolean_labeler(valid, 'gender')
test = prepare.boolean_labeler(test, 'gender')

In [8]:
# covered by new feature

train = prepare.boolean_labeler(train, 'partner')
valid = prepare.boolean_labeler(valid, 'partner')
test = prepare.boolean_labeler(test, 'partner')

In [9]:
# covered by new feature

train = prepare.boolean_labeler(train, 'dependents')
valid = prepare.boolean_labeler(valid, 'dependents')
test = prepare.boolean_labeler(test, 'dependents')

In [10]:
# covered by new feature

train = prepare.boolean_labeler(train, 'phone_service')
valid = prepare.boolean_labeler(valid, 'phone_service')
test = prepare.boolean_labeler(test, 'phone_service')

In [11]:
train = prepare.boolean_labeler(train, 'paperless_billing')
valid = prepare.boolean_labeler(valid, 'paperless_billing')
test = prepare.boolean_labeler(test, 'paperless_billing')

> Variables with Yes, No and a third option for not applicable: the third option will be converted to a 0 indicating no

In [12]:
def more_than_two_labels(df, col):
    df[f'{col}_enc'] = np.where(df[col] == 'No', '0',
                                np.where(df[col] == 'Yes', '1', '0'))
    return df

In [13]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'multiple_lines')
valid = prepare.more_than_two_labels(valid, 'multiple_lines')
test = prepare.more_than_two_labels(test, 'multiple_lines')

In [14]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'online_security')
valid = prepare.more_than_two_labels(valid, 'online_security')
test = prepare.more_than_two_labels(test, 'online_security')

In [15]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'online_backup')
valid = prepare.more_than_two_labels(valid, 'online_backup')
test = prepare.more_than_two_labels(test, 'online_backup')

In [16]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'device_protection')
valid = prepare.more_than_two_labels(valid, 'device_protection')
test = prepare.more_than_two_labels(test, 'device_protection')

In [17]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'tech_support')
valid = prepare.more_than_two_labels(valid, 'tech_support')
test = prepare.more_than_two_labels(test, 'tech_support')

In [18]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'streaming_tv')
valid = prepare.more_than_two_labels(valid, 'streaming_tv')
test = prepare.more_than_two_labels(test, 'streaming_tv')

In [19]:
# covered by new feature

train = prepare.more_than_two_labels(train, 'streaming_movies')
valid = prepare.more_than_two_labels(valid, 'streaming_movies')
test = prepare.more_than_two_labels(test, 'streaming_movies')

### Scale data

> Columns to be scaled: 

In [ ]:
# Identify columns to be scaled and decide on a method



### Create new features

#### Tenure converted to years

In [20]:
def months_to_years(df, col):
    df[f'{col}_months'] = (df[col] / 12).round(2)
    return df

In [21]:
train = prepare.months_to_years(train, 'tenure')
valid = prepare.months_to_years(valid, 'tenure')
test = prepare.months_to_years(test, 'tenure')

#### Single variable representing the information from phone_service and multiple_lines

In [22]:
telco.phone_service.value_counts(), telco.multiple_lines.value_counts()

(Yes    6361
 No      682
 Name: phone_service, dtype: int64, No                  3390
 Yes                 2971
 No phone service     682
 Name: multiple_lines, dtype: int64)

In [23]:
# If they have multiple lines, they get a 2, if they only have phone, they get a 1, else 0

def extra_lines(df):
    df['extra_lines'] = np.where(df['multiple_lines'] == 'Yes', '2',
                                np.where(df['multiple_lines'] == 'No', '1', '0'))
    return df

In [24]:
telco_test = extra_lines(telco)
telco_test.extra_lines.value_counts()

1    3390
2    2971
0     682
Name: extra_lines, dtype: int64

In [25]:
train = prepare.extra_lines(train)
valid = prepare.extra_lines(valid)
test = prepare.extra_lines(test)

#### Single variable representing info from dependents and partner

In [26]:
telco.dependents.value_counts(), telco.partner.value_counts()

(No     4933
 Yes    2110
 Name: dependents, dtype: int64, No     3641
 Yes    3402
 Name: partner, dtype: int64)

In [27]:
# If they have neither partner or dependents, they get 0
# If they have partner but no dependents, they get 1
# If they have partner and dependents, they get 2
# If they have only dependents, they get a 3 
## --> basically, the higher the score the harder it is for them to support their family

def family_support(df):
    df['family_support'] = np.where( (df['partner'] == 'No') & (df['dependents'] == 'Yes'), 3,
                                    np.where( (df['partner'] == 'Yes') & (df['dependents'] == 'Yes'), 2,
                                             np.where( (df['partner'] == 'Yes') & (df['dependents'] == 'No'), 1, 0)))
    return df

In [28]:
telco_test = family_support(telco)
telco_test.family_support.value_counts()

0    3280
2    1749
1    1653
3     361
Name: family_support, dtype: int64

In [29]:
train = prepare.family_support(train)
valid = prepare.family_support(valid)
test = prepare.family_support(test)

#### other ways to merge variables, such as streaming_tv & streaming_movies, online_security & online_backup

> Add new column indicating whether or not they have internet service, to cover the third option I removed

In [30]:
telco.internet_service_type.value_counts(), telco.internet_service_type_id.value_counts()

(Fiber optic    3096
 DSL            2421
 None           1526
 Name: internet_service_type, dtype: int64, 2    3096
 1    2421
 3    1526
 Name: internet_service_type_id, dtype: int64)

In [31]:
def has_internet(df):
    df['has_internet'] = np.where(df.internet_service_type_id == 3, 0, 1)
    return df

In [32]:
telco_test = has_internet(telco)
telco_test.has_internet.value_counts()

1    5517
0    1526
Name: has_internet, dtype: int64

In [33]:
train = prepare.has_internet(train)
valid = prepare.has_internet(valid)
test = prepare.has_internet(test)

> Add new column indicating how many internet services the customer has

In [38]:
# There are 6 service options
# If they have no internet, they get 0
# If they have internet but no extra services, they get 1
# If they have x extra services, they get 2 - 7

def internet_services(df):
    df['internet_services'] = (df.has_internet + 
                               df.online_security_enc + 
                               df.online_backup_enc + 
                               df.tech_support_enc + 
                               df.streaming_tv_enc + 
                               df.streaming_movies_enc + 
                               df.device_protection_enc)
    return df

In [39]:
telco_test = prepare.more_than_two_labels(telco, 'online_security')
telco_test = prepare.more_than_two_labels(telco, 'online_backup')
telco_test = prepare.more_than_two_labels(telco, 'tech_support')
telco_test = prepare.more_than_two_labels(telco, 'streaming_tv')
telco_test = prepare.more_than_two_labels(telco, 'streaming_movies')
telco_test = prepare.more_than_two_labels(telco, 'device_protection')

telco_test = internet_services(telco)
telco_test.internet_services.value_counts()

0    1526
4    1118
3    1033
2     966
5     852
1     693
6     571
7     284
Name: internet_services, dtype: int64

In [40]:
train = prepare.internet_services(train)
valid = prepare.internet_services(valid)
test = prepare.internet_services(test)

> Columns to be dropped from df due to new features and encoded features: 

In [42]:
train.columns.tolist()

['contract_type_id',
 'internet_service_type_id',
 'payment_type_id',
 'customer_id',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'tenure',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'monthly_charges',
 'total_charges',
 'churn',
 'payment_type',
 'internet_service_type',
 'contract_type',
 'gender_enc',
 'partner_enc',
 'dependents_enc',
 'phone_service_enc',
 'paperless_billing_enc',
 'multiple_lines_enc',
 'online_security_enc',
 'online_backup_enc',
 'device_protection_enc',
 'tech_support_enc',
 'streaming_tv_enc',
 'streaming_movies_enc',
 'tenure_months',
 'extra_lines',
 'family_support',
 'has_internet',
 'internet_services']

In [37]:
# [['gender', 'partner', 'partner_enc', 'dependents', 'dependents_enc', 'internet_service_type', 
#  'internet_service_type_id', 'online_security', 'online_security_enc', 'online_backup', 
#  'online_backup_enc', 'tech_support', 'tech_support_enc', 'streaming_tv', 'streaming_tv_enc',
#  'streaming_movies', 'streaming_movies_enc', 'phone_service', 'device_protection', 'device_protection_enc']]